<a href="https://colab.research.google.com/github/altinodantas/slides/blob/main/Regress%C3%A3o_log%C3%ADstica_com_Python_4Linux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regressão logística com Python

Para esta aula, estaremos trabalhando com o [Conjunto de dados do Titanic da Kaggle](https://www.kaggle.com/c/titanic). Este é um conjunto de dados muito famoso e muitas vezes é o primeiro passo em Machine Learning.

Vamos tentar montar um algoritmo de classificação (sobrevivente ou falecido) utilizando regressão logística no Python.

Usaremos uma versão "semi-limpa" do conjunto de dados do Titanic. Se você usar o conjunto de dados hospedado diretamente no Kaggle, talvez seja necessário fazer uma limpeza adicional não mostrada neste notebook.

## Importar bibliotecas
Vamos importar algumas bibliotecas para começar!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
! git clone https://github.com/leonardoamorim/datasets

## Os dados

Vamos começar lendo o arquivo titanic_train.csv em um DataFrame pandas.

In [ ]:
train = pd.read_csv('datasets/titanic_train.csv')

In [ ]:
train.head()

In [ ]:
train['Embarked']

In [ ]:
! wc -l datasets/titanic_train.csv

In [ ]:
train.columns

In [ ]:
train.info()

Survived (Sobreviveu): 0 = Não, 1 = Sim

Pclass (Classe): Classe de ingresso 1 = 1º, 2 = 2º, 3 = 3º

Sex (Sexo): Sexo do passageiro

Age (Idade): Idade em anos

Sibsp: Quantidade de irmãos / cônjuges a bordo do Titanic

Parch: Quantidade de pais / crianças a bordo do Titanic

Ticket (Bilhete): Número do bilhete de embarque

Fare (Tarifa): Tarifa paga pelo Passageiro

Cabin (Cabine): Número de cabine

Embarked (Embarque): Porto de Embarque (C = Cherbourg, Q=Queenstown, S = Southampton)

# Análise exploratória de dados 

Vamos começar algumas análises de dados exploratórios. Começaremos por verificar os dados que faltam!

## Dados ausentes

Podemos usar seaborn para criar um mapa de calor simples para ver onde estamos perdendo dados!

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='cividis');

Aproximadamente 20% dos dados de idade estão faltando. A proporção de idade que falta é provavelmente pequena o suficiente para que possamos fazer uma substituição razoável com alguma forma de imputação de dados. Olhando para a coluna Cabin, porém, parece que estamos perdendo muito desses dados para fazermos o mesmo. Provavelmente vamos descartar isso mais tarde ou mudá-lo para outro "Cabin Conhecido: 1 ou 0"

Continuemos visualizando mais alguns dos dados!

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')

In [ ]:
train['Age'].hist(bins=30,color='darkred',alpha=0.7)

In [ ]:
sns.countplot(x='SibSp',data=train)

In [ ]:
train['Fare'].hist(color='green',bins=40,figsize=(8,4))

___
## Limpando os dados

Queremos preencher dados de idade faltantes, em vez de simplesmente deixar cair as linhas de dados de idade que faltam. Uma maneira de fazer isso é preenchendo a idade média de todos os passageiros (imputação).
No entanto, podemos ser mais inteligentes sobre isso e verificar a idade média pela classe de passageiros. Por exemplo:


In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

Podemos ver os passageiros presumidamente mais ricos (que estão nas classes superiores) tendem a ser mais velhos, o que faz sentido. Usaremos esses valores de idade média para imputar com base em Pclass for Age.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

Agora aplique essa função!

In [ ]:
train['Age'] = train[['Age','Pclass']].apply(impute_age, axis=1)

Agora vamos verificar esse mapa de calor novamente!

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
train.columns

Ótimo! Vamos seguir em frente e deletar a coluna Cabin e a linha em Embarked que falta dado.

In [ ]:
train.drop('Cabin',axis=1,inplace=True)

In [ ]:
train['Embarked'].value_counts()

In [ ]:
train.head()

In [ ]:
train.dropna(inplace=True)

In [ ]:
train['Embarked'].value_counts()

## Convertendo recursos categóricos

Precisamos converter características categóricas em variáveis dummy usando pandas! Caso contrário, nosso algoritmo de Machine Learning não será capaz de aceitar esses recursos diretamente como entradas.

In [ ]:
train.info()

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first=True)  # drop_first=True > Para evitar a multi-colinearidade
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
embark

In [ ]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
sex

In [ ]:
train.head()

In [ ]:
train = pd.concat([train,sex,embark],axis=1)

In [ ]:
train.head()

Ótimo! Nossos dados estão prontos para o nosso modelo

# Construindo um modelo de Regressão Logística

Vamos começar dividindo nossos dados em um conjunto de treinamento e conjunto de testes (há outro arquivo titanic_test.csv que você pode usar ao invés, caso queira usar todos esses dados para treinar).

## Divisão treino-teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1), 
                                                    train['Survived'], test_size=0.30, 
                                                    random_state=101)

In [ ]:
X_test

## Treinamento e inferência

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression(max_iter=500)
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

Vamos seguir em frente para avaliar o nosso modelo!

## Avaliação

Podemos verificar a precisão, o recall e a pontuação f1 usando o relatório de classificação!

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

Não foi tão ruim! Você pode querer explorar outros recursos no outro arquivo titanic_text.csv. Algumas sugestões:

* Tente pegar o título (Dr., Sr., Sra., Etc.) do nome como parâmetro.
* Talvez a cabine possa ser uma característica.
* Existe alguma informação que você pode obter do bilhete?

In [ ]:
logmodel.predict([[5,	3, 35.0, 0, 0, 8.0500, 1,	0,	1]])
# PassengerId	Pclass Age SibSp Parch Fare	male Q S

## Testando com o KNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)

In [ ]:
knn.fit(X_train,y_train)

In [ ]:
predictions = knn.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
import pandas as pd                              
s = pd.Series(["A", "B", "C", "D", "E"],
              index=[1, 2, 3, 4, 5])

In [ ]:
s